In [1]:
from pyspark.sql.functions import explode, col,lit
import requests
import json

StatementMeta(, 90cd43f4-8313-460f-bccc-ef5791405251, 3, Finished, Available, Finished)

In [2]:
# Set dynamic partition overwrite mode
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

StatementMeta(, 90cd43f4-8313-460f-bccc-ef5791405251, 4, Finished, Available, Finished)

In [3]:
# Function to get category IDs
def get_category_ids():
    response = requests.get("https://paralleldemo.azurewebsites.net/api/LoadCategories")
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to load categories")

StatementMeta(, 90cd43f4-8313-460f-bccc-ef5791405251, 5, Finished, Available, Finished)

In [4]:
# Get list of category IDs
categories = get_category_ids()

# Initialize an empty list to hold DataFrames
dfCategories=spark.createDataFrame(categories)

DAG = {
    "activities": [
        {
            "name": f"category{row['categoryId']}", 
            "path": "Demo 4 - Load Orders", 
            "timeoutPerCellInSeconds": 4000,
            "args": {"categoryId": row["categoryId"] }
        }
        for row in dfCategories.rdd.collect()
                ],
        "concurrency": 0
    }

StatementMeta(, 90cd43f4-8313-460f-bccc-ef5791405251, 6, Finished, Available, Finished)

In [5]:
print(DAG)

StatementMeta(, 90cd43f4-8313-460f-bccc-ef5791405251, 7, Finished, Available, Finished)

{'activities': [{'name': 'category7', 'path': 'Demo 4 - Load Orders', 'timeoutPerCellInSeconds': 4000, 'args': {'categoryId': 7}}, {'name': 'category5', 'path': 'Demo 4 - Load Orders', 'timeoutPerCellInSeconds': 4000, 'args': {'categoryId': 5}}, {'name': 'category6', 'path': 'Demo 4 - Load Orders', 'timeoutPerCellInSeconds': 4000, 'args': {'categoryId': 6}}, {'name': 'category16', 'path': 'Demo 4 - Load Orders', 'timeoutPerCellInSeconds': 4000, 'args': {'categoryId': 16}}, {'name': 'category25', 'path': 'Demo 4 - Load Orders', 'timeoutPerCellInSeconds': 4000, 'args': {'categoryId': 25}}, {'name': 'category17', 'path': 'Demo 4 - Load Orders', 'timeoutPerCellInSeconds': 4000, 'args': {'categoryId': 17}}, {'name': 'category18', 'path': 'Demo 4 - Load Orders', 'timeoutPerCellInSeconds': 4000, 'args': {'categoryId': 18}}, {'name': 'category35', 'path': 'Demo 4 - Load Orders', 'timeoutPerCellInSeconds': 4000, 'args': {'categoryId': 35}}, {'name': 'category20', 'path': 'Demo 4 - Load Orders',

In [6]:
result=mssparkutils.notebook.runMultiple(DAG)

StatementMeta(, 90cd43f4-8313-460f-bccc-ef5791405251, 8, Finished, Available, Finished)

In [7]:
dfFinal=None
for category in result.keys():
    dfResult=spark.sql(f"select * from {category}")
    if dfFinal==None:
        dfFinal=dfResult
    else:
        dfFinal.unionByName(dfResult)


dfFinal.write.partitionBy("CategoryId").mode("overwrite").saveAsTable("Orders")

StatementMeta(, 90cd43f4-8313-460f-bccc-ef5791405251, 9, Finished, Available, Finished)